In [0]:
%sql
CREATE OR REPLACE TABLE dad_open_data.news.all_marketplace_news AS
SELECT  row_number() OVER (ORDER BY url) as id,
        id as local_id,
        'bbc' as source,
        CASE WHEN try_parse_url(url, 'HOST') IS NULL THEN NULL ELSE 'VALID' END as try_parse_url,
        url,
        author,
        headline,
        topics,
        try_cast(replace(publication_date, '"', '') as timestamp) as try_parse_publication_date,
        publication_date,
        null as updated_last,
        null as description,
        content,
        videos,
        images,
        related_articles,
        keyword,
        null as country,
        ai_query( 'databricks-meta-llama-3-3-70b-instruct',
                  'Identifica il country code piu probabile, rispondi solo col codice country code : ' || content
                ) as try_parse_country
FROM marketplace_news.datasets.bbc_news
UNION ALL
SELECT  row_number() OVER (ORDER BY url) + 1500 as id,
        id as local_id,
        'cnn' as source,
        CASE WHEN try_parse_url(url, 'HOST') IS NULL THEN NULL ELSE 'VALID' END as try_parse_url,
        url,
        author,
        headline,
        topics,
        try_cast(replace(publication_date, '"', '') as timestamp) as try_parse_publication_date,
        publication_date,
        updated_last,
        null as description,
        content,
        videos,
        images,
        related_articles,
        keyword,
        null as country,
        ai_query( 'databricks-meta-llama-3-3-70b-instruct',
                  'Identifica il country code piu probabile, rispondi solo col codice country code : ' || content
                ) as try_parse_country
FROM marketplace_news.datasets.cnn_news
UNION ALL
SELECT  row_number() OVER (ORDER BY url) + 3000 as id,
        null as local_id,
        'google' as source,
        CASE WHEN try_parse_url(url, 'HOST') IS NULL THEN NULL ELSE 'VALID' END as try_parse_url,
        url,
        publisher as author,
        title as headline,
        category as topics,
        try_cast(replace(date, '"', '') as timestamp) as try_parse_publication_date,
        date as publication_date,
        null as updated_last,
        null as description,
        null as content,
        null as videos,
        null as images,
        null as related_articles,
        keyword,
        country,
        country AS try_parse_country
FROM marketplace_news.datasets.google_news
UNION ALL
SELECT  row_number() OVER (ORDER BY url) + 4500 as id,
        id as local_id,
        'reuters' as source,
        CASE WHEN try_parse_url(url, 'HOST') IS NULL THEN NULL ELSE 'VALID' END as try_parse_url,
        url,
        author,
        headline,
        topics,
        try_cast(replace(publication_date, '"', '') as timestamp) as try_parse_publication_date,
        publication_date,
        updated_last,
        description,
        content,
        videos,
        images,
        related_articles,
        keyword,
        null as country,
        ai_query( 'databricks-meta-llama-3-3-70b-instruct',
                  'Identifica il country code piu probabile, rispondi solo col codice country code : ' || content
                ) as try_parse_country
FROM marketplace_news.datasets.reuters_news

In [0]:
%sql
select  count(*)
from    marketplace_news.datasets.bbc_news

In [0]:
%sql
select * from 
(
    SELECT  id as local_id,
            'bbc' as source,
            try_parse_url(url, 'HOST') as try_parse_url,
            url,
            author,
            headline,
            topics,
            publication_date,
            null as updated_last,
            null as description,
            content,
            videos,
            images,
            related_articles,
            keyword,
            null as country,
            row_number() OVER (ORDER BY publication_date) as rownumber
    FROM marketplace_news.datasets.bbc_news
    WHERE try_parse_url(url, 'HOST') IS NULL
      -- AND NOT (id RLIKE '^[0-9]+$')
)

In [0]:
%sql

SELECT    source, 
          try_parse_url, 
          count(*)
FROM      dad_open_data.news.all_marketplace_news
GROUP by  source,
          try_parse_url 


In [0]:
%sql
select    try_cast(replace(publication_date, '"', '') as timestamp) as trycast,
          *
from      dad_open_data.news.all_marketplace_news
where     try_parse_url = 'VALID' 
  and     id = 1712
--   and     try_parse_publication_date is not null

In [0]:
%sql

SELECT    source, 
          try_parse_url, 
          startwith(url, 'http'),
          count(*)
FROM      dad_open_data.news.all_marketplace_news
GROUP by  source,
          try_parse_url,
          startwith(url, 'http')

In [0]:
%sql
SELECT   
          count(*)
FROM      dad_open_data.news.all_marketplace_news
where     startswith(url, 'http') = true